# Set-RscMssqlInstance
This will allow you to assign an SLA to a MSSQL Instance.

# Workflow
To set information about the SQL Server Database, you need to do the following
1. Use **Connect-RSC** to Connect to RSC
1. Use **Get-RscCluster** to get the cluster id that the SQL Server is registered. This is akin to the datacenter that the SQL Server is located in. While you can run the next command without the cluster ID, you may get more records back than expected. This is because your SQL Server Host could exist on 2 different cluster. Once as a replicated source and once as the replicated target. 
1. Use **Get-RscHost** to get the host id that the SQL Server is installed on. 
1. Use **Get-RscMssqlInstance** to get the instance id of the SQL Server based on the name of the instance and the cluster where it is located. 
1. Use **Set-RscMssqlInstance** to make a change to the configuration of Rubrik for a specific MSSQL Instance. 
1. Use **Disconnect-RSC** to disconnect from RSC

## Connect to RSC

In [1]:
Import-Module RubrikSecurityCloud
Connect-RSC


ClientId                                AuthenticationState SdkSchemaVersion ServerVersion
--------                                ------------------- ---------------- -------------
client|kB5og68XXrFlfXcvWhn3jlHdf1OKyJCB          AUTHORIZED v20240219-20     v20240219-20



## Get the Cluster ID

In [2]:
$ClusterName = "vault-r-london"
$RscCluster = Get-RscCluster -Name $ClusterName

## Get the Host Information

In [3]:
$HostName = "rp-sql19s-001.perf.rubrik.com"
$RscHost = Get-RscHost -Name $HostName -OsType Windows

## Get the Instance Information

In [4]:
$RscMssqlInstance = Get-RscMssqlInstance -RscHost $RscHost -RscCluster $RscCluster

## Get the Database Information

In [5]:
$DatabaseName = "AdventureWorks2019"
$RscMssqlDatabase = Get-RscMssqlDatabase -Name $DatabaseName  -RscMssqlInstance $RscMssqlInstance
$RscMssqlDatabase


AuthorizedOperations            : 
ObjectType                      : MSSQL
SlaAssignment                   : DERIVED
ConfiguredSlaDomain             : 
EffectiveRetentionSlaDomain     : 
EffectiveSlaDomain              : RubrikSecurityCloud.Types.GlobalSlaReply
PendingSla                      : 
ReplicatedObjects               : 
CdmId                           : 42527305-5e85-4777-8689-955237d3d951
CdmLink                         : 
CdmOnDemandSnapshotCount        : 
CopyOnly                        : False
DagId                           : 9024703f-4510-5209-b3bb-52b7115ad5b9
HasLogConfigFromSla             : True
HasPermissions                  : True
HostLogRetention                : -1
Id                              : a3ca658f-3a80-5a3f-8108-a2156e5fd6a3
IsInAvailabilityGroup           : False
IsLogShippingSecondary          : False
IsMount                         : False
IsOnline                        : True
IsRelic                         : False
LogBackupFrequencyInSeconds   

## Change the configuration of Rubrik for the database
### Set DO NOT PROTECT on a database
This is used to tell Rubrik to not backup a database

In [6]:
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase -RscCluster $RscCluster -DoNotProtect -ExistingSnapshotRetention RETAIN_SNAPSHOTS


Items
-----
{}



### Set "Clear Existing assignment"
This is used to tell Rubrik clear the direct assignment and revert back to the SLA assignment from a higher level. Effectively reverses the DO NOT PROTECT setting. 

In [7]:
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase -RscCluster $RscCluster -ClearExistingProtection


Items
-----
{}



### Set an SLA to the database
This should only be used when the database needs a specific SLA that differs from the Host/Instance assigned SLA

In [8]:
$RscSlaDomain = Get-RscSla -Name "lumnah test"
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase -RscCluster $RscCluster -RscSlaDomain $RscSlaDomain


Items
-----
{}



As part of the assigning of the SLA, there are various options to set in addition to the SLA Domain.
#### Configuring Log Backups
Enabling log backups ensures that Point In Time Recovery is available for a database. 

In [9]:
$RscSlaDomain = Get-RscSla -Name "lumnah test"
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase -RscCluster $RscCluster -RscSlaDomain $RscSlaDomain -EnableLogBackups -UseSLALogConfig


Items
-----
{}



In [10]:
$RscSlaDomain = Get-RscSla -Name "lumnah test"
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase -RscCluster $RscCluster -RscSlaDomain $RscSlaDomain -EnableLogBackups -logBackupFrequencyInSeconds 3600 -logRetentionHours 168


Items
-----
{}



#### Pause Backups

In [11]:
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase -RscCluster $RscCluster -isPaused 


Items
-----
{77E231CA-551B-4E6D-8B2F-5FE0FF93504F}



#### Resume Backups

In [12]:
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase -RscCluster $RscCluster -isPaused:$false


Items
-----
{77E231CA-551B-4E6D-8B2F-5FE0FF93504F}



#### Configure Host Log Retention

In [13]:
$RscSlaDomain = Get-RscSla -Name "lumnah test"
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase -RscCluster $RscCluster -RscSlaDomain $RscSlaDomain -EnableLogBackups -EnableHostLogRetention -HostLogRetentionInSeconds 3600


Items
-----
{}



#### Disable Host Log Retention

In [14]:
$RscSlaDomain = Get-RscSla -Name "lumnah test"
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase -RscCluster $RscCluster -RscSlaDomain $RscSlaDomain -EnableLogBackups


Items
-----
{}



#### Set Pre-Script

In [15]:
$PreBackupScriptPath = "c:\rubrik\scripts\dosomething.bat"
$PreBackupScriptTimeoutMs = 5000
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase `
    -RscCluster $RscCluster `
    -PreBackupScriptErrorAction "SCRIPT_ERROR_ACTION_ABORT" `
    -PreBackupScriptPath $PreBackupScriptPath `
    -PreBackupScriptTimeoutMs $PreBackupScriptTimeoutMs


Items
-----
{77E231CA-551B-4E6D-8B2F-5FE0FF93504F}



#### Set Post-Script

In [16]:
$PostBackupScriptPath = "c:\rubrik\scripts\dosomething.bat"
$PostBackupScriptTimeoutMs = 5000
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase `
    -RscCluster $RscCluster `
    -PostBackupScriptErrorAction "SCRIPT_ERROR_ACTION_CONTINUE" `
    -PostBackupScriptPath $PostBackupScriptPath `
    -PostBackupScriptTimeoutMs $PostBackupScriptTimeoutMs


Items
-----
{77E231CA-551B-4E6D-8B2F-5FE0FF93504F}



#### Remove Pre Scipt
**Currently this will not work as the endpoint does not allow for it to be removed**

In [ ]:
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase `
    -RscCluster $RscCluster `
    -RemovePreBackupScript

#### Remove Post Scipt
**Currently this will not work as the endpoint does not allow for it to be removed**

In [ ]:
Set-RscMssqlDatabase -RscMssqlDatabase $RscMssqlDatabase `
    -RscCluster $RscCluster `
    -RemovePostBackupScript

---

# Disconnect from RSC
It is important to have each script you create end by disconnecting from RSC. If you do not, then you will run into issues with token authentication exhaustion. RSC allows a Service Account to have 10 concurrent open connections to RSC. Additional requests will be denied until existing connections are closed. 

You can easily disconnect your connection by running the below statement. 

In [17]:
Disconnect-Rsc

The Rubrik Security Cloud session has been terminated.
